In [56]:
from sklearn.datasets import make_classification

In [57]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [58]:
X, y = make_classification(n_samples=1000, n_features=20, n_informative=2, n_redundant=2, random_state=0)
X.shape, y.shape

((1000, 20), (1000,))

In [59]:
dataset = (X, y)


In [60]:
from sklearn.tree import DecisionTreeClassifier

In [61]:
model = DecisionTreeClassifier(max_depth=3).fit(X, y)
model

DecisionTreeClassifier(max_depth=3)

In [62]:
from generate_sql_query import convert_tree_to_json, generate_sql_query

tree_json = convert_tree_to_json(model)
print(tree_json)

{
  "feature_index": 11,
  "threshold": -0.1806,
  "left": {
    "feature_index": 11,
    "threshold": -0.4623,
    "left": {
      "feature_index": 19,
      "threshold": 1.927,
      "left": {
        "class": 0
      },
      "right": {
        "class": 1
      }
    },
    "right": {
      "feature_index": 19,
      "threshold": 1.1794,
      "left": {
        "class": 1
      },
      "right": {
        "class": 0
      }
    }
  },
  "right": {
    "feature_index": 12,
    "threshold": -1.4483,
    "left": {
      "feature_index": 11,
      "threshold": 0.4817,
      "left": {
        "class": 0
      },
      "right": {
        "class": 1
      }
    },
    "right": {
      "feature_index": 16,
      "threshold": -1.9313,
      "left": {
        "class": 1
      },
      "right": {
        "class": 1
      }
    }
  }
}


In [63]:
print(generate_sql_query(tree_json, features=[f"f{i}" for i in range(X.shape[1])]))

SELECT 
CASE WHEN f11 > -0.1806 THEN 
	CASE WHEN f11 > -0.4623 THEN 
		CASE WHEN f19 > 1.927 THEN 0
		ELSE 1
		END
	ELSE 
		CASE WHEN f19 > 1.1794 THEN 1
		ELSE 0
		END
	END
ELSE 
	CASE WHEN f12 > -1.4483 THEN 
		CASE WHEN f11 > 0.4817 THEN 0
		ELSE 1
		END
	ELSE 
		CASE WHEN f16 > -1.9313 THEN 1
		ELSE 1
		END
	END
END as class_label


In [64]:
import json
example = {
  "feature_index": 3,
  "threshold": -0.301,
  "left": {
    "class": 1
  },
  "right": {
    "feature_index": 1,
    "threshold": 1.4003,
    "left": {
      "class": 0
    },
    "right": {
      "class": 1
    }
  }
}
example_json = json.dumps(example)
example_sql = """SELECT
  CASE
    WHEN feature_3 > -0.301 THEN
      CASE
        WHEN feature_1 > 1.4003 THEN 1
        ELSE 0
      END
    ELSE 1
  END AS class_label
"""


print(generate_sql_query(example_json, features=[f"feature_{i}" for i in range(X.shape[1])]))


SELECT 
CASE WHEN feature_3 > -0.301 THEN 
	CASE WHEN feature_1 > 1.4003 THEN 1
	ELSE 0
	END
ELSE 1
END as class_label
